In [2]:
import numpy as np
import pandas as pd
from aequilibrae.paths.public_transport import HyperpathGenerating
from numba import jit

RS = 124  # random seed

In [3]:
def create_vertices(n):
    x = np.linspace(0, 1, n)
    y = np.linspace(0, 1, n)
    xv, yv = np.meshgrid(x, y, indexing="xy")
    vertices = pd.DataFrame()
    vertices["x"] = xv.ravel()
    vertices["y"] = yv.ravel()
    return vertices

n = 10
vertices = create_vertices(n)
vertices

,x,y
0,0.000000,0.0
1,0.111111,0.0
2,0.222222,0.0
3,0.333333,0.0
4,0.444444,0.0
...,...,...
95,0.555556,1.0
96,0.666667,1.0
97,0.777778,1.0
98,0.888889,1.0


In [4]:
@jit
def create_edges_numba(n):
    m = 2 * n * (n - 1)
    tail = np.zeros(m, dtype=np.uint32)
    head = np.zeros(m, dtype=np.uint32)
    k = 0
    for i in range(n - 1):
        for j in range(n):
            tail[k] = i + j * n
            head[k] = i + 1 + j * n
            k += 1
            tail[k] = j + i * n
            head[k] = j + (i + 1) * n
            k += 1
    return tail, head

def create_edges(n, seed=124):
    tail, head = create_edges_numba(n)
    edges = pd.DataFrame()
    edges["tail"] = tail
    edges["head"] = head
    m = len(edges)
    rng = np.random.default_rng(seed=seed)
    edges["trav_time"] = rng.uniform(0.0, 1.0, m)
    edges["delay_base"] = rng.uniform(0.0, 1.0, m)
    return edges

In [5]:
edges = create_edges(n, seed=RS)
edges

,tail,head,trav_time,delay_base
0,0,1,0.785253,0.287917
1,0,10,0.785859,0.970429
2,10,11,0.969136,0.854512
3,1,11,0.748060,0.319959
4,20,21,0.655551,0.047771
...,...,...,...,...
175,87,97,0.025590,0.642800
176,88,89,0.677292,0.482416
177,88,98,0.096483,0.138376
178,98,99,0.484049,0.136400


In [6]:
alpha = 10.0

delay_base = edges.delay_base.values
indices = np.where(delay_base == 0.0)
delay_base[indices] = 1.0 # use this to prevent an error?
freq_base = 1.0 / delay_base
freq_base[indices] = np.inf
edges["freq_base"] = freq_base

if alpha == 0.0:
    edges["freq"] = np.inf
else:
    edges["freq"] = edges.freq_base / alpha

In [7]:
edges

,tail,head,trav_time,delay_base,freq_base,freq
0,0,1,0.785253,0.287917,3.473221,0.347322
1,0,10,0.785859,0.970429,1.030472,0.103047
2,10,11,0.969136,0.854512,1.170259,0.117026
3,1,11,0.748060,0.319959,3.125397,0.312540
4,20,21,0.655551,0.047771,20.933209,2.093321
...,...,...,...,...,...,...
175,87,97,0.025590,0.642800,1.555694,0.155569
176,88,89,0.677292,0.482416,2.072900,0.207290
177,88,98,0.096483,0.138376,7.226708,0.722671
178,98,99,0.484049,0.136400,7.331395,0.733140


In [8]:
# Spiess & Florian
sf = HyperpathGenerating(
    edges, tail="tail", head="head", trav_time="trav_time", freq="freq"
)
sf._edges

,tail,head,trav_time,freq,edge_idx
0,0,1,0.785253,0.347322,0
1,0,10,0.785859,0.103047,1
2,10,11,0.969136,0.117026,2
3,1,11,0.748060,0.312540,3
4,20,21,0.655551,2.093321,4
...,...,...,...,...,...
175,87,97,0.025590,0.155569,175
176,88,89,0.677292,0.207290,176
177,88,98,0.096483,0.722671,177
178,98,99,0.484049,0.733140,178


In [9]:
sf._edges['head'].value_counts().sum()

180

In [10]:
sf._edges['head'].value_counts().sort_index().head(15)

head
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    2
12    2
13    2
14    2
15    2
Name: count, dtype: int64

In [11]:
sf._edges.head(15)

,tail,head,trav_time,freq,edge_idx
0,0,1,0.785253,0.347322,0
1,0,10,0.785859,0.103047,1
2,10,11,0.969136,0.117026,2
3,1,11,0.748060,0.312540,3
4,20,21,0.655551,2.093321,4
5,2,12,0.938885,0.275786,5
6,30,31,0.178614,0.238375,6
7,3,13,0.588647,0.105657,7
8,40,41,0.442799,0.107033,8
9,4,14,0.348847,0.130498,9


In [12]:
edge_check = sf._edges
edge_check[edge_check.edge_idx == 20]

,tail,head,trav_time,freq,edge_idx
20,1,2,0.945353,2.611624,20


In [13]:
vertex_count = sf._edges[['tail', 'head']].max().max() + 1
vertex_values = pd.DataFrame(list(range(0,vertex_count+1)), columns=['vertex'])

head_v_count = pd.DataFrame(sf._edges['head'].value_counts().sort_index().cumsum()).reset_index()
head_v_count.columns = ['vertex', 'count_cumsum']

vertex_values = pd.merge(vertex_values, head_v_count, on=['vertex'], how='left')
vertex_values = vertex_values['count_cumsum'].fillna(method = 'ffill').fillna(0)
vertex_values = vertex_values.shift(1).fillna(0)
indptr_check = vertex_values.to_numpy().astype(np.uint32)

indptr_check

C:\Users\germa\AppData\Local\Temp\ipykernel_17480\655044509.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vertex_values = vertex_values['count_cumsum'].fillna(method = 'ffill').fillna(0)


array([  0,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,
        14,  16,  18,  20,  22,  24,  26,  28,  29,  31,  33,  35,  37,
        39,  41,  43,  45,  47,  48,  50,  52,  54,  56,  58,  60,  62,
        64,  66,  67,  69,  71,  73,  75,  77,  79,  81,  83,  85,  86,
        88,  90,  92,  94,  96,  98, 100, 102, 104, 105, 107, 109, 111,
       113, 115, 117, 119, 121, 123, 124, 126, 128, 130, 132, 134, 136,
       138, 140, 142, 143, 145, 147, 149, 151, 153, 155, 157, 159, 161,
       162, 164, 166, 168, 170, 172, 174, 176, 178, 180], dtype=uint32)

In [14]:
sf._edges['head'].value_counts().sort_index()

head
1     1
2     1
3     1
4     1
5     1
     ..
95    2
96    2
97    2
98    2
99    2
Name: count, Length: 99, dtype: int64

In [15]:
# cumsum of the count of ordered head vertex
sf._indptr

array([  0,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  12,
        14,  16,  18,  20,  22,  24,  26,  28,  29,  31,  33,  35,  37,
        39,  41,  43,  45,  47,  48,  50,  52,  54,  56,  58,  60,  62,
        64,  66,  67,  69,  71,  73,  75,  77,  79,  81,  83,  85,  86,
        88,  90,  92,  94,  96,  98, 100, 102, 104, 105, 107, 109, 111,
       113, 115, 117, 119, 121, 123, 124, 126, 128, 130, 132, 134, 136,
       138, 140, 142, 143, 145, 147, 149, 151, 153, 155, 157, 159, 161,
       162, 164, 166, 168, 170, 172, 174, 176, 178, 180], dtype=uint32)

In [16]:
np.all(sf._indptr == indptr_check)

True

In [17]:
# sort head, edge_idx and retrieve edge_idx
sf._edge_idx

array([  0,  20,  40,  60,  80, 100, 120, 140, 160,   1,   2,   3,   5,
        22,   7,  42,   9,  62,  11,  82,  13, 102,  15, 122,  17, 142,
        19, 162,  21,   4,  23,  24,  25,  27,  44,  29,  64,  31,  84,
        33, 104,  35, 124,  37, 144,  39, 164,  41,   6,  43,  26,  45,
        46,  47,  49,  66,  51,  86,  53, 106,  55, 126,  57, 146,  59,
       166,  61,   8,  63,  28,  65,  48,  67,  68,  69,  71,  88,  73,
       108,  75, 128,  77, 148,  79, 168,  81,  10,  83,  30,  85,  50,
        87,  70,  89,  90,  91,  93, 110,  95, 130,  97, 150,  99, 170,
       101,  12, 103,  32, 105,  52, 107,  72, 109,  92, 111, 112, 113,
       115, 132, 117, 152, 119, 172, 121,  14, 123,  34, 125,  54, 127,
        74, 129,  94, 131, 114, 133, 134, 135, 137, 154, 139, 174, 141,
        16, 143,  36, 145,  56, 147,  76, 149,  96, 151, 116, 153, 136,
       155, 156, 157, 159, 176, 161,  18, 163,  38, 165,  58, 167,  78,
       169,  98, 171, 118, 173, 138, 175, 158, 177, 178, 179],
 

In [18]:
sf._edges.sort_values(by=['head', 'edge_idx'])

,tail,head,trav_time,freq,edge_idx
0,0,1,0.785253,0.347322,0
20,1,2,0.945353,2.611624,20
40,2,3,0.639182,0.450637,40
60,3,4,0.895527,0.201273,60
80,4,5,0.307495,0.115251,80
...,...,...,...,...,...
175,87,97,0.025590,0.155569,175
158,97,98,0.510875,0.122883,158
177,88,98,0.096483,0.722671,177
178,98,99,0.484049,0.733140,178


In [19]:
check_edge_idx = sf._edges.sort_values(by=['head', 'edge_idx']).edge_idx.to_numpy()
check_edge_idx 

array([  0,  20,  40,  60,  80, 100, 120, 140, 160,   1,   2,   3,   5,
        22,   7,  42,   9,  62,  11,  82,  13, 102,  15, 122,  17, 142,
        19, 162,  21,   4,  23,  24,  25,  27,  44,  29,  64,  31,  84,
        33, 104,  35, 124,  37, 144,  39, 164,  41,   6,  43,  26,  45,
        46,  47,  49,  66,  51,  86,  53, 106,  55, 126,  57, 146,  59,
       166,  61,   8,  63,  28,  65,  48,  67,  68,  69,  71,  88,  73,
       108,  75, 128,  77, 148,  79, 168,  81,  10,  83,  30,  85,  50,
        87,  70,  89,  90,  91,  93, 110,  95, 130,  97, 150,  99, 170,
       101,  12, 103,  32, 105,  52, 107,  72, 109,  92, 111, 112, 113,
       115, 132, 117, 152, 119, 172, 121,  14, 123,  34, 125,  54, 127,
        74, 129,  94, 131, 114, 133, 134, 135, 137, 154, 139, 174, 141,
        16, 143,  36, 145,  56, 147,  76, 149,  96, 151, 116, 153, 136,
       155, 156, 157, 159, 176, 161,  18, 163,  38, 165,  58, 167,  78,
       169,  98, 171, 118, 173, 138, 175, 158, 177, 178, 179], d

In [20]:
np.all(check_edge_idx == sf._edge_idx)

True

In [23]:
sf._skim_cols[:].shape

(180,)

In [21]:
dest = n * n - 1
sf.run(origin=0, destination=dest, volume=1.0)

: 

In [46]:
edges_df = sf._edges

for i in range(sf._indptr[dest], sf._indptr[dest+1]):
    print(i)
    edge_idx = sf._edge_idx[i]
    print(edge_idx)
    print(sf._edges.trav_time[edge_idx])
    display(edges_df[edges_df.edge_idx == edge_idx])
edges_df[edges_df['head'] == 99]

178
178
0.48404915578440333


,tail,head,trav_time,freq,edge_idx,volume
178,98,99,0.484049,0.73314,178,0.88162


179
179
0.7912608054550185


,tail,head,trav_time,freq,edge_idx,volume
179,89,99,0.791261,0.149404,179,0.11838


,tail,head,trav_time,freq,edge_idx,volume
178,98,99,0.484049,0.733140,178,0.88162
179,89,99,0.791261,0.149404,179,0.11838


In [47]:
edges_df[edges_df.edge_idx == 179]

,tail,head,trav_time,freq,edge_idx,volume
179,89,99,0.791261,0.149404,179,0.11838


In [48]:
sf._edges

,tail,head,trav_time,freq,edge_idx,volume
0,0,1,0.785253,0.347322,0,0.771194
1,0,10,0.785859,0.103047,1,0.228806
2,10,11,0.969136,0.117026,2,0.000000
3,1,11,0.748060,0.312540,3,0.771194
4,20,21,0.655551,2.093321,4,0.216982
...,...,...,...,...,...,...
175,87,97,0.025590,0.155569,175,0.000000
176,88,89,0.677292,0.207290,176,0.000000
177,88,98,0.096483,0.722671,177,0.240592
178,98,99,0.484049,0.733140,178,0.881620


In [49]:
sf.u_i_vec

array([52.09625924, 50.7362477 , 50.29012703, 52.41375548, 50.82758145,
       50.11924603, 48.24846199, 48.39241413, 45.92053915, 43.88370516,
       43.54334338, 46.78859487, 45.72524603, 43.6762203 , 43.16977658,
       43.76784005, 43.34089596, 44.08791865, 43.09076564, 41.84234352,
       39.93075556, 38.8407916 , 38.09956231, 36.68694389, 34.69013688,
       35.00111374, 35.2733203 , 36.01293616, 35.79890705, 38.39658043,
       38.26240408, 37.22094869, 32.99118395, 32.62201292, 29.4599568 ,
       26.0790963 , 29.324187  , 28.81208559, 27.06670372, 31.05404947,
       30.83352321, 33.22772334, 32.64916301, 31.18388494, 27.07531288,
       24.71102132, 25.811869  , 21.14912348, 19.87453072, 26.68571608,
       29.8690225 , 27.20246995, 26.85435153, 25.74804623, 23.16957413,
       20.45977989, 22.49318111, 19.93196034, 18.63722004, 15.88485328,
       41.1443306 , 30.15607543, 25.93814664, 22.81153912, 21.51407745,
       17.89385237, 17.52250986, 14.27434694, 14.15979309, 13.32

In [50]:
sf.skim_i_vec

AttributeError: 'HyperpathGenerating' object has no attribute 'skim_i_vec'

In [26]:
edges_df.loc[edges_df['head'] == 99]

,tail,head,trav_time,freq,edge_idx,volume
178,98,99,0.484049,0.733140,178,0.88162
179,89,99,0.791261,0.149404,179,0.11838


#### Replicate SF run PART 1

In [26]:
sf._edges

,tail,head,trav_time,freq,edge_idx
0,0,1,0.785253,0.347322,0
1,0,10,0.785859,0.103047,1
2,10,11,0.969136,0.117026,2
3,1,11,0.748060,0.312540,3
4,20,21,0.655551,2.093321,4
...,...,...,...,...,...
175,87,97,0.025590,0.155569,175
176,88,89,0.677292,0.207290,176
177,88,98,0.096483,0.722671,177
178,98,99,0.484049,0.733140,178
